In [94]:
import os
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings(action = 'ignore')
from datetime import datetime
import pickle

import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop=set(stopwords.words('english'))

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, SpatialDropout1D, GlobalMaxPool1D, Dropout, CuDNNLSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.metrics import roc_auc_score

In [3]:
sd = random.randint(0,99999999)
print(sd)

np.random.seed(sd)
random.seed(sd)
os.environ['PYTHONHASHEED']=str(sd)

config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(sd)

sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)

tf.logging.set_verbosity(tf.logging.ERROR)

86369728


In [4]:
train = pd.read_csv('C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/train.csv')

In [53]:
def text_preprocessing(text_list):
    
    bigram_list = []
    
    for text in text_list:
        txt = re.sub('[^a-z]', ' ', text.lower())
        token = word_tokenize(txt)
        bigram = [token[i] + '.' + token[i+1] for i in range(len(token) - 1)]
        bigram_list.append(' '.join(bigram))
        
    return bigram_list

In [54]:
train['bigram'] = text_preprocessing(train.text)

In [67]:
pd.Series([len(x.split()) for x in train['bigram']]).describe()

count    7613.000000
mean       16.421384
std         6.067671
min         0.000000
25%        12.000000
50%        17.000000
75%        21.000000
max        32.000000
dtype: float64

In [66]:
def text2sequence(train_text, max_len=1000):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len)
    return X_train, vocab_size

In [69]:
train_y = df_train['target']
train_X, vocab_size = text2sequence(df_train['bigram'], max_len = 32)
print(train_X.shape, train_y.shape)

vocab_size :  2174
(300, 32) (300,)


In [70]:
auc_ = 0
def auc_score(y_true, y_pred):
    global auc_
    try:
        auc_ = roc_auc_score( y_true, y_pred, average='macro', sample_weight = None).astype('float32')
    except ValueError:
        pass
    return auc_

def auc(y_true, y_pred):
    score = tf.py_func( lambda y_true, y_pred : auc_score(y_true, y_pred) , [y_true, y_pred], 'float32', stateful = False, name = 'sklearnAUC' )
    return score

In [95]:
def BiLSTM(vocab_size, max_len=1000):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length = max_len))
    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(64, activation='tanh', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc])
    model.summary()
    return model

In [96]:
print('START TIME: ', datetime.now().isoformat())
model = BiLSTM(vocab_size, max_len=32)
early_stopping = EarlyStopping(patience=1000, min_delta=0.00001)
history = model.fit(train_X, train_y, epochs=1000, batch_size=128, validation_split=0.3, callbacks=[early_stopping])

print('END TIME: ', datetime.now().isoformat())

START TIME:  2020-02-25T15:05:25.890817
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 32, 128)           278272    
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 32, 128)           0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 32, 128)           99328     
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 128)           0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 32, 128)           99328     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_16 (Den

Epoch 49/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0618 - auc: 0.9978 - val_loss: 1.7289 - val_auc: 0.6883
Epoch 50/1000
210/210 [==============================] - 0s 204us/sample - loss: 0.0605 - auc: 1.0000 - val_loss: 1.7348 - val_auc: 0.6878
Epoch 51/1000
210/210 [==============================] - 0s 218us/sample - loss: 0.0609 - auc: 0.9983 - val_loss: 1.7287 - val_auc: 0.6883
Epoch 52/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0604 - auc: 0.9962 - val_loss: 1.7243 - val_auc: 0.6888
Epoch 53/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0594 - auc: 0.9970 - val_loss: 1.7230 - val_auc: 0.6893
Epoch 54/1000
210/210 [==============================] - 0s 205us/sample - loss: 0.0582 - auc: 0.9996 - val_loss: 1.7131 - val_auc: 0.6893
Epoch 55/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0574 - auc: 0.9997 - val_loss: 1.7067 - val_auc: 0.6903
Epoch 56/1000
210/210 [====

Epoch 108/1000
210/210 [==============================] - 0s 209us/sample - loss: 0.0207 - auc: 1.0000 - val_loss: 1.4198 - val_auc: 0.7130
Epoch 109/1000
210/210 [==============================] - 0s 204us/sample - loss: 0.0216 - auc: 0.9997 - val_loss: 1.5058 - val_auc: 0.7155
Epoch 110/1000
210/210 [==============================] - 0s 209us/sample - loss: 0.0205 - auc: 0.9999 - val_loss: 1.5886 - val_auc: 0.7166
Epoch 111/1000
210/210 [==============================] - 0s 212us/sample - loss: 0.0215 - auc: 1.0000 - val_loss: 1.6345 - val_auc: 0.7233
Epoch 112/1000
210/210 [==============================] - 0s 204us/sample - loss: 0.0183 - auc: 1.0000 - val_loss: 1.6591 - val_auc: 0.7238
Epoch 113/1000
210/210 [==============================] - 0s 198us/sample - loss: 0.0202 - auc: 1.0000 - val_loss: 1.6721 - val_auc: 0.7263
Epoch 114/1000
210/210 [==============================] - 0s 209us/sample - loss: 0.0207 - auc: 1.0000 - val_loss: 1.6317 - val_auc: 0.7263
Epoch 115/1000
210/2

210/210 [==============================] - 0s 209us/sample - loss: 0.0124 - auc: 1.0000 - val_loss: 1.8521 - val_auc: 0.6569
Epoch 167/1000
210/210 [==============================] - 0s 204us/sample - loss: 0.0119 - auc: 1.0000 - val_loss: 1.8367 - val_auc: 0.6595
Epoch 168/1000
210/210 [==============================] - 0s 195us/sample - loss: 0.0129 - auc: 1.0000 - val_loss: 1.7979 - val_auc: 0.6615
Epoch 169/1000
210/210 [==============================] - 0s 195us/sample - loss: 0.0118 - auc: 1.0000 - val_loss: 1.7526 - val_auc: 0.6631
Epoch 170/1000
210/210 [==============================] - 0s 200us/sample - loss: 0.0116 - auc: 1.0000 - val_loss: 1.7212 - val_auc: 0.6641
Epoch 171/1000
210/210 [==============================] - 0s 200us/sample - loss: 0.0111 - auc: 1.0000 - val_loss: 1.6961 - val_auc: 0.6641
Epoch 172/1000
210/210 [==============================] - 0s 200us/sample - loss: 0.0118 - auc: 1.0000 - val_loss: 1.6725 - val_auc: 0.6656
Epoch 173/1000
210/210 [===========

210/210 [==============================] - 0s 200us/sample - loss: 0.0091 - auc: 1.0000 - val_loss: 1.8427 - val_auc: 0.6548
Epoch 225/1000
210/210 [==============================] - 0s 212us/sample - loss: 0.0095 - auc: 1.0000 - val_loss: 1.8518 - val_auc: 0.6548
Epoch 226/1000
210/210 [==============================] - 0s 200us/sample - loss: 0.0090 - auc: 1.0000 - val_loss: 1.8783 - val_auc: 0.6528
Epoch 227/1000
210/210 [==============================] - 0s 202us/sample - loss: 0.0093 - auc: 0.9999 - val_loss: 1.8937 - val_auc: 0.6533
Epoch 228/1000
210/210 [==============================] - 0s 200us/sample - loss: 0.0093 - auc: 0.9999 - val_loss: 1.9104 - val_auc: 0.6528
Epoch 229/1000
210/210 [==============================] - 0s 207us/sample - loss: 0.0093 - auc: 1.0000 - val_loss: 1.9308 - val_auc: 0.6528
Epoch 230/1000
210/210 [==============================] - 0s 209us/sample - loss: 0.0093 - auc: 0.9999 - val_loss: 1.9351 - val_auc: 0.6517
Epoch 231/1000
210/210 [===========

...

210/210 [==============================] - 0s 207us/sample - loss: 0.0073 - auc: 1.0000 - val_loss: 1.8634 - val_auc: 0.6872
Epoch 631/1000
210/210 [==============================] - 0s 218us/sample - loss: 0.0072 - auc: 0.9999 - val_loss: 1.8595 - val_auc: 0.6867
Epoch 632/1000
210/210 [==============================] - 0s 228us/sample - loss: 0.0075 - auc: 1.0000 - val_loss: 1.8588 - val_auc: 0.6872
Epoch 633/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0074 - auc: 1.0000 - val_loss: 1.8542 - val_auc: 0.6878
Epoch 634/1000
210/210 [==============================] - 0s 223us/sample - loss: 0.0071 - auc: 1.0000 - val_loss: 1.8512 - val_auc: 0.6883
Epoch 635/1000
210/210 [==============================] - 0s 223us/sample - loss: 0.0073 - auc: 1.0000 - val_loss: 1.8504 - val_auc: 0.6872
Epoch 636/1000
210/210 [==============================] - 0s 232us/sample - loss: 0.0071 - auc: 1.0000 - val_loss: 1.8574 - val_auc: 0.6867
Epoch 637/1000
210/210 [===========

210/210 [==============================] - 0s 214us/sample - loss: 0.0070 - auc: 1.0000 - val_loss: 1.8350 - val_auc: 0.6903
Epoch 689/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0070 - auc: 1.0000 - val_loss: 1.8396 - val_auc: 0.6908
Epoch 690/1000
210/210 [==============================] - 0s 205us/sample - loss: 0.0073 - auc: 1.0000 - val_loss: 1.8464 - val_auc: 0.6903
Epoch 691/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0068 - auc: 1.0000 - val_loss: 1.8504 - val_auc: 0.6908
Epoch 692/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0073 - auc: 1.0000 - val_loss: 1.8488 - val_auc: 0.6914
Epoch 693/1000
210/210 [==============================] - 0s 218us/sample - loss: 0.0074 - auc: 1.0000 - val_loss: 1.8407 - val_auc: 0.6924
Epoch 694/1000
210/210 [==============================] - 0s 209us/sample - loss: 0.0075 - auc: 0.9999 - val_loss: 1.8413 - val_auc: 0.6924
Epoch 695/1000
210/210 [===========

210/210 [==============================] - 0s 204us/sample - loss: 0.0072 - auc: 0.9997 - val_loss: 1.8780 - val_auc: 0.6944
Epoch 747/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0068 - auc: 1.0000 - val_loss: 1.8964 - val_auc: 0.6944
Epoch 748/1000
210/210 [==============================] - 0s 216us/sample - loss: 0.0071 - auc: 1.0000 - val_loss: 1.9123 - val_auc: 0.6939
Epoch 749/1000
210/210 [==============================] - 0s 223us/sample - loss: 0.0071 - auc: 1.0000 - val_loss: 1.9147 - val_auc: 0.6939
Epoch 750/1000
210/210 [==============================] - 0s 226us/sample - loss: 0.0070 - auc: 1.0000 - val_loss: 1.9140 - val_auc: 0.6939
Epoch 751/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0070 - auc: 1.0000 - val_loss: 1.9136 - val_auc: 0.6944
Epoch 752/1000
210/210 [==============================] - 0s 221us/sample - loss: 0.0074 - auc: 0.9997 - val_loss: 1.9009 - val_auc: 0.6944
Epoch 753/1000
210/210 [===========

210/210 [==============================] - 0s 218us/sample - loss: 0.0076 - auc: 1.0000 - val_loss: 1.9361 - val_auc: 0.6960
Epoch 805/1000
210/210 [==============================] - 0s 218us/sample - loss: 0.0072 - auc: 1.0000 - val_loss: 1.9385 - val_auc: 0.6960
Epoch 806/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0069 - auc: 1.0000 - val_loss: 1.9304 - val_auc: 0.6960
Epoch 807/1000
210/210 [==============================] - 0s 223us/sample - loss: 0.0074 - auc: 0.9999 - val_loss: 1.9193 - val_auc: 0.6955
Epoch 808/1000
210/210 [==============================] - 0s 218us/sample - loss: 0.0071 - auc: 1.0000 - val_loss: 1.9116 - val_auc: 0.6955
Epoch 809/1000
210/210 [==============================] - 0s 214us/sample - loss: 0.0069 - auc: 1.0000 - val_loss: 1.9021 - val_auc: 0.6952
Epoch 810/1000
210/210 [==============================] - 0s 223us/sample - loss: 0.0072 - auc: 1.0000 - val_loss: 1.8902 - val_auc: 0.6950
Epoch 811/1000
210/210 [===========

KeyboardInterrupt: 